# Exercise - VaR

## Data

This problem uses `weekly` return data from `data/spx_returns_weekly.xlsx`.

Choose any `4` stocks to evaluate below.

For example, 
* `AAPL`
* `META`
* `NVDA`
* `TSLA`

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm

# TICKS = ["AMD", "ADBE", "ORCL", "BKNG"]
TICKS = ["AAPL", "META", "NVDA", "TSLA"]

stock_rets = pd.read_excel("../data/spx_returns_weekly.xlsx", sheet_name="s&p500 rets").set_index("date")[TICKS]
bench_rets = pd.read_excel("../data/spx_returns_weekly.xlsx", sheet_name="benchmark rets").set_index("date")

stock_rets.head()

,AAPL,META,NVDA,TSLA
date,,,,
2015-01-09,0.024514,-0.009055,-0.009315,-0.057685
2015-01-16,-0.053745,-0.032931,0.000836,-0.065760
2015-01-23,0.065950,0.035255,0.037578,0.042575
2015-01-30,0.036997,-0.024669,-0.072636,0.011476
2015-02-06,0.019114,-0.018967,0.062269,0.067589


In [2]:
bench_rets.head()

,SPY,BTC,USO,TLT,IEF,IYR,GLD
date,,,,,,,
2015-01-09,-0.005744,-0.079179,-0.080945,0.029453,0.013517,0.029953,0.027875
2015-01-16,-0.012827,-0.281115,0.002735,0.016175,0.010188,0.019471,0.044858
2015-01-23,0.016565,0.137612,-0.072559,0.011863,0.001558,0.007958,0.013957
2015-01-30,-0.026931,-0.030969,0.048235,0.026044,0.011992,-0.013361,-0.006279
2015-02-06,0.030584,-0.027431,0.092593,-0.051020,-0.022724,-0.013173,-0.038963


# 1 Diversification

## 1.1

Using the full sample, calculate for each series the (unconditional) 
* volatility
* empirical VaR (.05)
* empirical CVaR (.05)

Recall that by **empirical** we refer to the direct quantile estimation. (For example, using `.quantile()` in pandas.

## 1.2
Form an equally-weighted portfolio of the investments.

Calculate the statistics of `1.1` for this portfolio, and compare the results to the individual return statistics. What do you find? What is driving this result?

## 1.3
Re-calculate `1.2`, but this time drop your most volatile asset, and replace the portion it was getting with 0. (You could imagine we're replacing the most volatile asset with a negligibly small risk-free rate.)

In comparing the answer here to 1.2, how much risk is your most volatile asset adding to the portfolio? Is this in line with the amount of risk we measured in the stand-alone risk-assessment of `1.1`?

In [3]:
# 1.1
def compute_stock_stats(rets: pd.DataFrame) -> pd.DataFrame:
    vol = rets.std()
    eVaR = rets.quantile(0.05)
    eCVaR = rets[rets <= rets.quantile(0.05)].mean()

    return pd.DataFrame(data = {
        "vol": vol,
        "5% VaR": eVaR,
        "5% CVaR": eCVaR,
    })

stats = compute_stock_stats(stock_rets)
stats.head()

,vol,5% VaR,5% CVaR
AAPL,0.038362,-0.056366,-0.083125
META,0.048722,-0.070012,-0.103196
NVDA,0.064246,-0.086853,-0.116455
TSLA,0.081323,-0.117397,-0.147814


In [4]:
# 1.2
n = len(TICKS)
stock_rets['portfolio'] = stock_rets[TICKS].sum(axis=1) / n

pf_stats = compute_stock_stats(stock_rets).loc["portfolio"]
pf_stats.head()

vol        0.043758
5% VaR    -0.061950
5% CVaR   -0.084992
Name: portfolio, dtype: float64

In [5]:
# 1.3
most_volatile = stats.sort_values(by="vol").iloc[-1]
print(most_volatile)

ticks_minus_most_vol = TICKS[:]
ticks_minus_most_vol.remove(most_volatile.name)
np_rets = stock_rets[ticks_minus_most_vol]
np_rets['portfolio'] = stock_rets[ticks_minus_most_vol].sum(axis=1) / n

np_stats = compute_stock_stats(np_rets).loc["portfolio"]
np_stats.head()

vol        0.081323
5% VaR    -0.117397
5% CVaR   -0.147814
Name: TSLA, dtype: float64


/var/folders/7x/mk6b_27178v6y3m4bk5dnbkc0000gn/T/ipykernel_57661/3275957941.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  np_rets['portfolio'] = stock_rets[ticks_minus_most_vol].sum(axis=1) / n


vol        0.030282
5% VaR    -0.042463
5% CVaR   -0.060531
Name: portfolio, dtype: float64

The asset is adding approximately 0.1 in annualized volatility to the portfolio, which is approximately the difference between the asset-free portfolio's volatility and the asset's vol divided by 4. In any case, the portfolio volatility is still lower than all but 1 of the assets.

***

# 2. Dynamic Measures

## 2.1 

Let's measure the **conditional** statistics of the equally-weighted portfolio of `1.2`, as of the end of the sample.

#### Volatility
For each security, calculate the **rolling** volatility series, $\sigma_t$, with a window of $m=26$.

The value at $\sigma_t$ in the notes denotes the estimate using data through time $t-1$, and thus (potentially) predicting the volatility at $\sigma_{t}$. 

#### Mean
Suppose we can approximate that the daily mean return is zero.

#### VaR
Calculate the **normal VaR** and **normal CVaR** for $q=.05$ and $\tau=1$ as of the end of the sample. Use the approximation, $\texttt{z}_{.05} = -1.65$.

#### Notation Note
In this setup, we are using a forecasted volatility, $\sigma_t$ to estimate the VaR return we would have estimated at the end of $t-1$ in prediction of time $t$.

#### Conclude and Compare
Report
* volatility (annualized).
* normal VaR (.05)
* normal CVaR (.05)

How do these compare to the answers in `1.2`?

In [6]:
def compute_rolling_volatility(series: pd.Series, window=26):
    """
    Compute rolling volatility (annualized standard deviation) for a series.
    
    Args:
        series: pandas Series with datetime index
        window: number of periods for rolling window
    
    Returns:
        pandas Series with rolling volatility
    """
    # Calculate rolling standard deviation
    rolling_std = series.rolling(window=window).std(ddof=1)
    
    # Annualize (assuming daily data, multiply by sqrt(252))
    # Adjust multiplier based on your data frequency:
    # Daily: sqrt(252), Weekly: sqrt(52), Monthly: sqrt(12)
    # periods_per_year = 52  # adjust as needed
    rolling_vol = rolling_std # * np.sqrt(periods_per_year)
    
    return rolling_vol

rolling_vol = compute_rolling_volatility(stock_rets)
rolling_vol.tail()

,AAPL,META,NVDA,TSLA,portfolio
date,,,,,
2025-04-25,0.047947,0.054819,0.078598,0.097651,0.053597
2025-05-02,0.047564,0.057596,0.078556,0.096224,0.053211
2025-05-09,0.047741,0.057173,0.076363,0.077772,0.048479
2025-05-16,0.049628,0.057751,0.082727,0.084876,0.053749
2025-05-23,0.051479,0.058009,0.082962,0.083011,0.054049


In [7]:
# compute normal VaR
Z_05 = -1.65
mean = 0
sigma = rolling_vol['portfolio'].iloc[-1]
normal_VaR = sigma*Z_05
print(f"Normal distribution with rolling standard deviation 5% VaR: {normal_VaR:.4f}")

Normal distribution with rolling standard deviation 5% VaR: -0.0892


In [8]:
normal_cvar = -(norm.pdf(Z_05)/norm.cdf(Z_05))*sigma
print(f"Normal distribution with rolling standard deviation 5% CVaR: {normal_cvar:.4f}")

Normal distribution with rolling standard deviation 5% CVaR: -0.1117


## 2.2

Backtest the VaR using the **hit test**. Namely, check how many times the realized return at $t$ was smaller than the VaR return calculated using $\sigma_t$, (where again remember the notation in the notes uses $\sigma_t$ as a vol based on data through $t-1$.)

Report the percentage of "hits" using both the 
* expanding volatility
* rolling volatility

In [9]:
# expanding variance = 1/(t-1) * sum(r^2_i) to t-1 for each t
# exp_var = stock_rets.expanding().var()
exp_vol = stock_rets['portfolio'].expanding().std()
exp_VaRs = exp_vol * Z_05

# compute the percentage of returns that hit the predicted VaR boundaries
# estimated from expanding volatilities
perc_expanding = (stock_rets['portfolio'] <= exp_VaRs).astype(int).mean() * 100

print(f"% of hits using expanding vol: {perc_expanding:.4f}%")

% of hits using expanding vol: 4.9815%


In [10]:
# rolling variance = 1/window_size * sum^window_size (r^2_{t-i})
rol_vol = compute_rolling_volatility(stock_rets['portfolio'])
rol_VaRs = rol_vol * Z_05

perc_rolling = (stock_rets['portfolio'] <= rol_VaRs).astype(int).mean() * 100

print(f"% of hits using rolling vol: {perc_rolling:.4f}%")

% of hits using rolling vol: 3.6900%


***